# 常规赛：PALM病理性近视病灶检测与分割基线方案
本项目参考[红白黑](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/277680)大佬基线：[常规赛：PALM病理性近视病灶检测与分割基线方案 基线1.0版本](https://aistudio.baidu.com/aistudio/projectdetail/1941312)

**赛题简述**

	PALM病理性近视病灶检测与分割常规赛的重点是研究和发展与病理性近视诊断和患者眼底照片病变分割相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上检测病理性近视的自动算法。具体任务为：检测眼底图像是否出现视网膜萎缩病变和脱离病变，若有，需要实现病变区域的分割。
    
![](https://ai-studio-static-online.cdn.bcebos.com/bcb4c79dd34242e0bd83db8ea8062d7fa6a12a59999c44f09a73c9298a45fae3)
    
    
**数据基本标签**

	萎缩病变分割金标准：萎缩区域：0；背景：255；
	脱离病变分割金标准：脱离区域：0；背景：255。
    
**训练数据集**
文件名称：Train

Train里有两个文件夹，一个是fundus_images，一个是Lesion_Masks。

* fundus_images文件夹内包含**800**张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、N0001.jpg、P0001.jpg和V0001.jpg。

* Lesion_Masks文件夹内包含两个文件夹：**Atrophy**和**Detachment**，其中，Atrophy文件夹包含fundus_images里眼底彩照的萎缩病变区域分割金标准，大小与对应的眼底彩照一致。命名前缀与对应眼底图像一致，后缀为bmp。同理，Detachment文件夹包含fundus_images里眼底彩照的脱离病变区域分割金标准，大小与对应的眼底彩照一致，命名前缀与对应眼底图像一致，后缀为bmp。**请注意，若Lesion_Masks中无某张眼底图像的病灶分割结果，说明该图像中不包含对应的病灶区域**。

**测试数据集**

文件名称：PALM-Testing400-Images.zip

压缩包里包含400张眼底彩照，命名形如T0001.jpg。



# 一、基线内容方案

* 解压数据与数据划分

    -- # 解压数据集

    -- !unzip -oq /home/aistudio/data/data93538/常规赛：PALM病理性近视病灶检测与分割.zip -d PaddleSeg/data

    -- # 划分数据

    -- !python utils/dataset_splited.py

* 数据标签预处理

    -- # 转换标签

    -- !python utils/dataset_pretrans.py

    * 原分类为1分类问题，为了问题研究的充分性和更大程度上利用多分类间的类别竞争对分类结构有一个更好的指导

    * 二分类问题描述，原标签为0不变，将255无效值转换为1值

    * 后期提交前会后处理，消去1值，换回赛题需要的255值

* 利用PaddleSeg套件加速赛题开发与测试: 使用套件config中的模型yml进行快速高效的实验开发——注意数据集yml的配置

* 实现训练流程

* 实现预测流程

* 完成提交结果 -- 基线方案为0.67+的得分(D_iter:500, A_iter:2000)，可从训练迭代次数、损失函数、模型入手

    -- # 提交结果后处理

    -- utils/post_process.py
    

> 先解压需要的PadleSeg套件

In [ ]:
# 解压PaddleSeg压缩包
!unzip -oq data/data88946/PaddleSeg.zip -d /home/aistudio/
# 修改文件名
!mv PaddleSeg-release-v2.0 PaddleSeg

上一步mv，可以将PaddleSeg加压后的文件目录改成PaddleSeg
>PaddleSeg下载至github的release2.0版本，为了方便大家使用，已添加在了数据集中供大家使用

In [ ]:
# 删除data目录 —— 用于数据划分时，产生了意料之外的数据扩充时的数据重置
# %cd /home/aistudio/
# !rm -rf PaddleSeg/data
# 解压数据集到PaddleSeg目录下的data文件夹
!unzip -oq /home/aistudio/data/data93538/常规赛：PALM病理性近视病灶检测与分割.zip -d PaddleSeg/data

In [ ]:
# 查看数据集文件的树形结构
!tree -d PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割

# 二、比赛数据集情况

PALM-Testing400-Images : 测试数据集文件夹

Train : 训练数据集文件夹

* Lesion_Masks ; 标注图片

	-- Detachment 视网膜脱落标注 -- 样本较少，存在同时萎缩的样本
    
   -- Atrophy 萎缩标注
   
* fundus_image : 原始图片

> 注意没有验证数据集，这里提供一个简单的划分程序，划分比例为0.7

> utils/dataset_splited.py

通过PIL的Image读取图片查看以下原数据与Label标注情况

In [ ]:
from PIL import Image
import numpy as np
# 读取图片
png_img = Image.open('PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割/Train/fundus_image/H0003.jpg')
png_img  # 展示真实图片

In [ ]:
bmp_img = Image.open('PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Atrophy/H0003.bmp')
bmp_img   # 展示萎缩标注图片

In [ ]:
bmp_img = Image.open('PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Detachment/P0053.bmp')
bmp_img   # 展示脱落标注图片

# 三、划分数据集与数据预处置

当前划分比例为0.8——可在utils文件夹下的dataset_splited.py修改**train_percent**为其它值

数据预处置-可在utils文件夹下的dataset_pretrans.py中查看相关代码--实现将255转化为1，原问题变二分类问题

> 注意：当前数据处理中，存在对数据进行扩充，因此当前程序运行一次之后会使得原分类数据数目增加——(扩增不宜过大，否则划分数据进行训练和验证时会出现偏差问题)

> 仅可运行一次，多次运行会导致填充数量过多，这是源码中扩充机制决定的(直接扩充到源文件夹中，所以下一次划分的时候就会默认把源文件中所有的文件读取)

> 感兴趣可前往查看dataset_splited.py的数据扩充区

In [ ]:
# 保证路径为初始路径
%cd /home/aistudio

# 划分数据
!python utils/dataset_splited.py

# 转换标签--预处置
!python utils/dataset_pretrans.py

移除原数据，减小项目空间，减少下一次进入和退出保存时花的时间

In [ ]:
# 移除’常规赛：PALM病理性近视病灶检测与分割‘文件夹
!rm -rf PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割
!rm -rf PaddleSeg/data/__MACOSX 

# 四、选择比赛模型

基线模型为:  配置略微修改的`PaddleSeg/configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k.yml`

具体配置在
	
	-- example/unet_Deta.yml
    * 用于分割脱落情况
    
	-- example/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml
    * 用于分割萎缩情况

详细数据集配置在

	-- example/unet_Deta.yml
    * 用于分割脱落情况

	-- example/pascal_voc2012_Atro.yml
    * 用于设置萎缩情况数据
    

> 针对不同分割任务配置不同的分割模型，以适配不同的任务驱动

## 1.配置_base_中对应的数据集与模型

> 具体配置信息，可以在`examples`文件夹下查看相应`yml`文件，有相应的注释。

**简要说明`example/pascal_voc2012_Atro.yml`中的配置要点**，以方便大家修改其他的数据集yml适配模型训练

```yml
# 该文件需要自行移动到PaddleSeg/configs/_base_下, 并修改模型文件中的_base_路径(建议)
# 或者根据该文件中的train_dataset与val_dataset，对_base_下相应的yml进行修改

# 批大小   -- 可通过训练时动态调整
batch_size: 8
# 迭代次数 -- 可通过训练时动态调整 
iters: 40000

# 自定义数据集加载的方式
train_dataset:
  # 自定义数据集加载方式：Dataset
  type: Dataset
  # 数据集目录--当前项目中类别Detachment的数据都放在了这里：data/Detachment
  # 不同类别的训练，可以换成不同的数据根目录
  # Atrophy类--对应data/Atrophy
  dataset_root: data/Atrophy
  # 该目录下划分数据产生的txt：data/Detachment/train_list.txt
  train_path: data/Atrophy/train_list.txt
  # 类别--当前已转换为2分类问题
  num_classes: 2
  # 预处理
  transforms:
    - type: ResizeStepScaling
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    # 考虑到当前数据集正确划分区域较小，RandomPaddingCrop是否有必要不清楚，可以自行尝试
    # 下边有提供Resize处理缩放--选1缩放标准即可
    # - type: RandomPaddingCrop
    #   crop_size: [800, 800]
    - type: Resize
      target_size: [680, 680]
    - type: RandomHorizontalFlip
    - type: RandomVerticalFlip
    - type: Normalize
  # 数据集加载方式--注意要一一对应
  mode: train

val_dataset:
  type: Dataset
  dataset_root: data/Atrophy
  # 注意验证数据集的path和训练数据集path的区别
  val_path: data/Atrophy/val_list.txt
  num_classes: 2
  transforms:
    # 修改padding为Resize，padding仅作填充，对于大图片无法缩放
    - type: Resize
      target_size: [680, 680]
    - type: Normalize
  # mode务必对应，否则无法索引正确的路径
  mode: val


# train_dataset:
#   type: PascalVOC
#   dataset_root: data/VOCdevkit/
#   transforms:
#     - type: ResizeStepScaling
#       min_scale_factor: 0.5
#       max_scale_factor: 2.0
#       scale_step_size: 0.25
#     - type: RandomPaddingCrop
#       crop_size: [512, 512]
#     - type: RandomHorizontalFlip
#     - type: RandomDistort
#       brightness_range: 0.4
#       contrast_range: 0.4
#       saturation_range: 0.4
#     - type: Normalize
#   mode: train

# val_dataset:
#   type: PascalVOC
#   dataset_root: data/VOCdevkit/
#   transforms:
#     - type: Padding
#       target_size: [512, 512]
#     - type: Normalize
#   mode: val

# 以下参数可以在模型yml中被配置
# 优化器选择
optimizer:
  type: sgd
  momentum: 0.9
  # 正则化
  weight_decay: 4.0e-5

# 学习率--多项式
learning_rate:
  value: 0.01
  decay:
    type: poly
    power: 0.9
    end_lr: 0.0

# 损失配置项--可参考其它模型yml文件
loss:
  types:
    - type: CrossEntropyLoss
  coef: [1]
```

**至于模型配置，以当前使用Emanet为例说明**:

> 大家在使用模型yml时，为了保证数可读取，可以使用提供的两个数据集yml，分别操作加载不同的数据

```yml
# _base_: '../_base_/pascal_voc12aug.yml'

# 用调整后的pascal_voc2012.yml替换原始的_base_数据集配置文件
# 使用新模型要保证数据集路径正确
# 当前模型配置文件yml要移动到指定的PaddleSeg/paddleseg/emanet文件夹下
# 数据集要移动到PaddleSeg/paddleseg/_base_文件夹下
_base_: '../_base_/pascal_voc2012_Atro.yml'

model:
  type: EMANet
  backbone:
    type: ResNet50_vd
    output_stride: 8
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/resnet50_vd_ssld_v2.tar.gz
  ema_channels: 512
  gc_channels: 256
  num_bases: 64
  stage_num: 3
  momentum: 0.25
  concat_input: True
  enable_auxiliary_loss: True
  align_corners: True

optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 0.0005


loss:
  types:
    - type: CrossEntropyLoss
    - type: DiceLoss
  coef: [4.,2.]
```

> 将预置的基线配置yml移动到相应的文件夹下

* _base_: 数据加载yml存放

* emanet: emanet模型yml存放

In [ ]:
%cd /home/aistudio/
!cp -u example/pascal_voc2012_Atro.yml PaddleSeg/configs/_base_/
!cp -u example/pascal_voc2012_Deta.yml PaddleSeg/configs/_base_/
# !cp -u example/emanet_resnet50_os8_voc12aug_512x512_40k_Deta.yml PaddleSeg/configs/emanet/
!cp -u example/unet_Deta.yml PaddleSeg/configs/
!cp -u example/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml PaddleSeg/configs/emanet/


## 2.启动训练

### 0.下载依赖项

在平台上可以不用执行，环境支持；线下可能需要下载。

In [ ]:
# 下载依赖项，保证PaddleSeg正常运行
%cd PaddleSeg
%pwd
!pip install -r requirements.txt

### 1. 载入模型开始训练

> 更换自定义的模型文件时，只需要修改对应的模型yml、数据集yml(_base_中的yml)、以及替换下方的yml即可进行训练了

> 不要更改输出目录，否则后边的代码也需要修改，包括预测、后处理中的所有路径

In [ ]:
%cd ~
# %pwd
# 训练分割Detachment的模型，并保存到 ./output/Detachment
!python PaddleSeg/train.py --c PaddleSeg/configs/unet_Deta.yml \
--use_vdl \
--save_interval 200 \
--do_eval \
--seed 2021 \
--iters 1000 \
--learning_rate 0.01 \
--save_dir output/Detachment



In [ ]:
# 训练分割Atrophy的模型，并保存到/output/Atrophy
!python  PaddleSeg/train.py --c  PaddleSeg/configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml \
--use_vdl \
--save_interval 200 \
--do_eval \
--seed 2021 \
--iters 3000 \
--learning_rate 0.005 \
--save_dir output/Atrophy

## 3.开始预测

这可以直接使用emanet进行预测，不用修改数据集yml，也不用修改相应的模型yml，注意训练权重对应即可！

> 预测结果按类别分别放在`./output/result/Detachment` 和 `./output/result/Atrophy` 下

> 如果使用不同的模型对不同的分割类进行讨论，注意模型yml即可

> 提交结果为两种单独预测的结果

**默认使用Iou评估最好的模型训练参数--best_model**

In [15]:
%cd ~
# 预测Detachment
!python PaddleSeg/predict.py --config PaddleSeg/configs/unet_Deta.yml \
--model_path output/Detachment/best_model/model.pdparams \
--image_path PaddleSeg/data/Detachment/Test \
--save_dir PaddleSeg/output/result/Detachment


# # 预测Atrophy
# !python PaddleSeg/predict.py --config PaddleSeg/configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml  \
# --model_path output/Atrophy/best_model/model.pdparams \
# --image_path PaddleSeg/data/Atrophy/Test \
# --save_dir PaddleSeg/output/result/Atrophy

/home/aistudio
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-08-16 16:08:42 [INFO]	
---------------Config Information---------------
batch_size: 8
iters: 40000
learning_rate:
  decay:
    end_lr: 0.0
    power: 0.9
    type: poly
  value: 0.01
loss:
  coef:
  - 4.0
  - 2.0
  types:
  - type: CrossEntropyLoss
  - type: DiceLoss
model:
  align_corners: true
  backbone:
    output_stride: 8
    p

# 五、预测结果后处理

* 将类别值1换为255，进行赛题结果提交

In [ ]:
%cd /home/aistudio/
!python utils/post_process.py

# 六、提交比赛结果

In [ ]:
# 复制文件到最顶层目录
%cd /home/aistudio
!cp -r PaddleSeg/output/result/ Lesion_Segmentation

# 过程移动文件--保证不包含生成的子目录
%cd Lesion_Segmentation
!cp -r Detachment/pseudo_color_prediction/. Detachment
!cp -r Atrophy/pseudo_color_prediction/. Atrophy

# 获取指定的提交目录格式
!rm -rf Detachment/added_prediction
!rm -rf Detachment/pseudo_color_prediction

!rm -rf Atrophy/added_prediction
!rm -rf Atrophy/pseudo_color_prediction

# 压缩文件
%cd /home/aistudio
!zip -r Lesion_Segmentation.zip Lesion_Segmentation
# 删除复制的文件
!rm -rf Lesion_Segmentation

In [ ]:
# # 删除zip的文件--丢失提交结果，需重新后处理生成
# !rm -rf Lesion_Segmentation.zip
# # 删除预测结果--丢失预测结果，需重新预测
# !rm -rf PaddleSeg/output/result
# # 删除output文件夹--丢失模型参数，需重新训练
# !rm -rf PaddleSeg/output
# # 删除data文件夹--数据将丢失，需要重新解压，划分，预处置
# !rm -rf PaddleSeg/data

# 七、其它建议

> 对于少类别预测较差，可以通过更换模型、数据增广等方式获取更优的预测

* 1. 模型建议：注意力模型(EMANet等)或者调整unet模型
* 2. 损失建议：多损失结构，不同的coef，针对赛题的特殊损失等(Dice等)
* 3. 模型魔改建议：尝试对Unet添加注意力模块，修改参数，或者调整不同的backbone与indices组合
* 4. 优化器与学习率策略的调整

最后，祝大家Paddle越用越顺手，比赛越打越顺利——取得理想的成绩！